# Data and Preprocessing

## **Data Sources:**

We are using Yahoo Finance (yfinance) and Alpha Vantage as our primary data sources for stock market and financial data. These platforms are widely used in both academic research and professional financial analysis due to their accuracy and comprehensive datasets.

**Why These Sources Are Credible**
1. Yahoo Finance (yfinance)

- Yahoo Finance aggregates data from stock exchanges, company filings (such as SEC reports), and financial institutions.

- It is frequently cited in academic papers and financial industry reports as a reliable source for historical stock prices and company financials.

- The API has been tested by financial professionals and researchers, and its data is cross-referenced with exchange reports for accuracy.

2. Alpha Vantage

- Alpha Vantage partners with financial data providers and offers real-time and historical stock data, fundamental financials, and technical indicators.

- It is trusted by institutions and developers for algorithmic trading, portfolio management, and quantitative research.

- The API provides data directly from stock exchanges, ensuring real-time accuracy.

In [10]:
%%capture
pip install yfinance

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [12]:
import yfinance as yf
import pandas as pd

stock = 'GOOGL'
df = yf.download(stock, start="2010-01-01", end="2024-12-31")  # Exclude 2020 and include data till end of 2024

# Filter out data for 2020 (if any)
df = df[(df.index.year != 2020)]

# Reset index to make 'Date' a column
df.reset_index(inplace=True)

# Rename columns if needed
df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']


[*********************100%***********************]  1 of 1 completed


Below we calculate several technical indicators, they are as follows:

1. **Simple Moving Average (SMA)** – Computes the 50-day average closing price to smooth out short-term fluctuations and highlight long-term trends.

2. **Exponential Moving Average (EMA)** – Similar to SMA but gives more weight to recent prices, making it more responsive to recent market changes.

3. **Relative Strength Index (RSI)** – Measures the strength of recent price movements over a 14-day period to determine if a stock is overbought (high RSI) or oversold (low RSI).

4. **MACD (Moving Average Convergence Divergence)** – Uses the difference between two EMAs (12-day and 26-day) to identify potential buy or sell signals.

5. **Bollinger Bands** – Uses a 20-day moving average with upper and lower bands based on standard deviation to measure price volatility.

6. **Average True Range (ATR)** – Computes a 14-day average of true range (high-low and previous close differences) to measure market volatility.

7. **On-Balance Volume (OBV)** – Tracks cumulative volume flow to indicate whether volume is supporting price movements.

Finally, reset the DataFrame index and remove unnecessary columns.


In [13]:
# Simple Moving Average (SMA)
df['SMA_50'] = df['Close'].rolling(window=50).mean()

# Exponential Moving Average (EMA)
df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()

# Relative Strength Index (RSI)
delta = df['Close'].diff(1)
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
df['RSI_14'] = 100 - (100 / (1 + rs))

# MACD (Moving Average Convergence Divergence)
df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
df['EMA_26'] = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = df['EMA_12'] - df['EMA_26']

# Bollinger Bands
df['SMA_20'] = df['Close'].rolling(window=20).mean()
df['STD_20'] = df['Close'].rolling(window=20).std()
df['Upper_Band'] = df['SMA_20'] + (df['STD_20'] * 2)
df['Lower_Band'] = df['SMA_20'] - (df['STD_20'] * 2)

# Average True Range (ATR)
df['High-Low'] = df['High'] - df['Low']
df['High-PrevClose'] = abs(df['High'] - df['Close'].shift(1))
df['Low-PrevClose'] = abs(df['Low'] - df['Close'].shift(1))
df['TR'] = df[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
df['ATR_14'] = df['TR'].rolling(window=14).mean()

# On-Balance Volume (OBV)
df['OBV'] = (df['Volume'] * ((df['Close'].diff() > 0) * 2 - 1)).cumsum()

# Reset index to ensure it's a clean DataFrame
df.reset_index(inplace=True)
df = df.drop(columns=['level_0', 'index'], errors='ignore')

df.tail()

,Date,Close,High,Low,Open,Volume,SMA_50,EMA_50,RSI_14,EMA_12,...,SMA_20,STD_20,Upper_Band,Lower_Band,High-Low,High-PrevClose,Low-PrevClose,TR,ATR_14,OBV
3515,2024-12-23,194.406113,194.875573,189.931255,192.398415,25675000,174.700717,176.990717,71.498729,188.083719,...,181.315562,10.948753,203.213067,159.418056,4.944318,3.685769,1.258549,4.944318,5.998636,5817796052
3516,2024-12-24,195.884399,195.884399,193.557078,194.615856,10403300,175.326772,177.731646,70.668252,189.283823,...,182.746510,10.885822,204.518154,160.974865,2.327321,1.478287,0.849035,2.327321,5.890504,5828199352
3517,2024-12-26,195.375000,196.523671,194.156402,194.925505,12046600,175.932663,178.423542,72.328084,190.220928,...,184.078656,10.709326,205.497307,162.660005,2.367269,0.639272,1.727997,2.367269,5.793778,5816152752
3518,2024-12-27,192.538254,195.095322,190.430680,194.725737,18891400,176.487805,178.977060,67.337998,190.577439,...,185.263478,10.235210,205.733897,164.793059,4.664641,0.279678,4.944320,4.944320,5.917471,5797261352
3519,2024-12-30,191.020004,192.328495,188.902433,189.581658,14264700,177.057080,179.449333,64.890216,190.645526,...,186.386356,9.512947,205.412250,167.360461,3.426062,0.209759,3.635821,3.635821,5.990958,5782996652


The section below fetches quarterly financial data for a given stock ticker (e.g., AAPL) from Alpha Vantage using its API. It retrieves three key financial reports:

- **Income Statement** – Shows revenue, expenses, and profit over time.

- **Balance Sheet** – Provides a snapshot of assets, liabilities, and equity.

- **Cash Flow Statement** – Tracks cash movement in operations, investments, and financing.

In [14]:
import requests
import pandas as pd
import time

API_KEY = 'AP IKE Y'  # Replace with your API key
ticker = stock 

# Function to fetch quarterly financial data (Income Statement)
def get_quarterly_income_statement(ticker, api_key):
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={ticker}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()

    if 'quarterlyReports' in data:
        df_quarterly = pd.DataFrame(data['quarterlyReports'])
        df_quarterly['fiscalDateEnding'] = pd.to_datetime(df_quarterly['fiscalDateEnding'])
        return df_quarterly
    else:
        print("Income Statement Data not available.")
        return None

# Function to fetch quarterly balance sheet data
def get_quarterly_balance_sheet(ticker, api_key):
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={ticker}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()

    if 'quarterlyReports' in data:
        df_balance_sheet = pd.DataFrame(data['quarterlyReports'])
        df_balance_sheet['fiscalDateEnding'] = pd.to_datetime(df_balance_sheet['fiscalDateEnding'])
        return df_balance_sheet
    else:
        print("Balance Sheet Data not available.")
        return None

# Function to fetch quarterly cash flow data
def get_quarterly_cash_flow(ticker, api_key):
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={ticker}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()

    if 'quarterlyReports' in data:
        df_cash_flow = pd.DataFrame(data['quarterlyReports'])
        df_cash_flow['fiscalDateEnding'] = pd.to_datetime(df_cash_flow['fiscalDateEnding'])
        return df_cash_flow
    else:
        print("Cash Flow Data not available.")
        return None

# Fetch data
df_income_statement = get_quarterly_income_statement(ticker, API_KEY)
df_balance_sheet = get_quarterly_balance_sheet(ticker, API_KEY)
df_cash_flow = get_quarterly_cash_flow(ticker, API_KEY)

# Pause to prevent exceeding API rate limits
time.sleep(12)

# Display first few rows of the financial data
if df_income_statement is not None:
    print("\nQuarterly Income Statement:")
    print(df_income_statement.head())
    df_income_statement.to_csv("quarterly_income_statement.csv", index=False)

if df_balance_sheet is not None:
    print("\nQuarterly Balance Sheet:")
    print(df_balance_sheet.head())
    df_balance_sheet.to_csv("quarterly_balance_sheet.csv", index=False)

if df_cash_flow is not None:
    print("\nQuarterly Cash Flow:")
    print(df_cash_flow.head())
    df_cash_flow.to_csv("quarterly_cash_flow.csv", index=False)

print("\nCSV files have been successfully saved!")



Quarterly Income Statement:
  fiscalDateEnding reportedCurrency  grossProfit totalRevenue costOfRevenue  \
0       2025-03-31              USD  53873000000  90234000000   36361000000   
1       2024-12-31              USD  55856000000  96469000000   40613000000   
2       2024-09-30              USD  51794000000  88268000000   36474000000   
3       2024-06-30              USD  49235000000  84742000000   35507000000   
4       2024-03-31              USD  46827000000  80539000000   33712000000   

  costofGoodsAndServicesSold operatingIncome sellingGeneralAndAdministrative  \
0                36361000000     30606000000                      3539000000   
1                40613000000     30972000000                      4405000000   
2                36474000000     28521000000                      3599000000   
3                35507000000     27425000000                      3158000000   
4                33712000000     25472000000                      3026000000   

  researchAndDe

In [15]:
# Convert necessary columns to numeric
df_balance_sheet['totalAssets'] = pd.to_numeric(df_balance_sheet['totalAssets'], errors='coerce')
df_balance_sheet['totalCurrentAssets'] = pd.to_numeric(df_balance_sheet['totalCurrentAssets'], errors='coerce')
df_balance_sheet['totalCurrentLiabilities'] = pd.to_numeric(df_balance_sheet['totalCurrentLiabilities'], errors='coerce')
df_balance_sheet['totalShareholderEquity'] = pd.to_numeric(df_balance_sheet['totalShareholderEquity'], errors='coerce')
df_balance_sheet['shortTermDebt'] = pd.to_numeric(df_balance_sheet['shortTermDebt'], errors='coerce')
df_balance_sheet['longTermDebt'] = pd.to_numeric(df_balance_sheet['longTermDebt'], errors='coerce')
df_balance_sheet['commonStockSharesOutstanding'] = pd.to_numeric(df_balance_sheet['commonStockSharesOutstanding'], errors='coerce')
#df_balance_sheet['marketPrice'] = pd.to_numeric(df_balance_sheet['marketPrice'], errors='coerce')  # Assuming you have marketPrice for P/E Ratio

df_income_statement['netIncome'] = pd.to_numeric(df_income_statement['netIncome'], errors='coerce')

# Convert fiscalDateEnding to datetime
df_balance_sheet['fiscalDateEnding'] = pd.to_datetime(df_balance_sheet['fiscalDateEnding'], errors='coerce')
df_income_statement['fiscalDateEnding'] = pd.to_datetime(df_income_statement['fiscalDateEnding'], errors='coerce')

# Calculate EPS and P/E Ratio
df_income_statement['EPS'] = df_income_statement['netIncome'] / df_balance_sheet['commonStockSharesOutstanding']
#df_income_statement['PE_Ratio'] = df_balance_sheet['marketPrice'] / df_income_statement['EPS']

# Calculate D/E Ratio (Debt-to-Equity)
df_balance_sheet['Total_Debt'] = df_balance_sheet['shortTermDebt'] + df_balance_sheet['longTermDebt']
df_balance_sheet['DE_Ratio'] = df_balance_sheet['Total_Debt'] / df_balance_sheet['totalShareholderEquity']

# Calculate Current Ratio
df_balance_sheet['Current_Ratio'] = df_balance_sheet['totalCurrentAssets'] / df_balance_sheet['totalCurrentLiabilities']

# Calculate ROA (Return on Assets)
df_income_statement['ROA'] = df_income_statement['netIncome'] / df_balance_sheet['totalAssets']

# Calculate ROE (Return on Equity)
df_income_statement['ROE'] = df_income_statement['netIncome'] / df_balance_sheet['totalShareholderEquity']

# Now merge these ratios with the original financial data
df_combined = df_balance_sheet[['fiscalDateEnding', 'totalAssets', 'totalCurrentAssets', 'totalCurrentLiabilities', 'totalShareholderEquity', 'shortTermDebt', 'longTermDebt', 'commonStockSharesOutstanding']].copy()

df_combined = df_combined.merge(df_income_statement[['fiscalDateEnding', 'netIncome']], on='fiscalDateEnding', how='left')

# Merge the calculated ratios back into the dataframe
df_combined['EPS'] = df_combined['netIncome'] / df_combined['commonStockSharesOutstanding']
#df_combined['PE_Ratio'] = df_combined['netIncome'] / df_combined['EPS']
df_combined['Total_Debt'] = df_combined['shortTermDebt'] + df_combined['longTermDebt']
df_combined['DE_Ratio'] = df_combined['Total_Debt'] / df_combined['totalShareholderEquity']
df_combined['Current_Ratio'] = df_combined['totalCurrentAssets'] / df_combined['totalCurrentLiabilities']
df_combined['ROA'] = df_combined['netIncome'] / df_combined['totalAssets']
df_combined['ROE'] = df_combined['netIncome'] / df_combined['totalShareholderEquity']

# Display the combined data with ratios


In [16]:
# Convert 'fiscalDateEnding' in df_combined to match the 'Date' format in df
df_combined['Date'] = pd.to_datetime(df_combined['fiscalDateEnding'])

# Merge the dataframes on Date
# We'll use forward fill to carry forward fundamental data until next quarter report
merged_df = pd.merge_asof(df.sort_values('Date'),
                         df_combined.sort_values('Date'),
                         on='Date',
                         direction='forward')

# Drop duplicate columns if any
merged_df = merged_df.drop(columns=['fiscalDateEnding'], errors='ignore')

# Display the merged dataframe
print(merged_df.tail())

           Date       Close        High         Low        Open    Volume  \
3515 2024-12-23  194.406113  194.875573  189.931255  192.398415  25675000   
3516 2024-12-24  195.884399  195.884399  193.557078  194.615856  10403300   
3517 2024-12-26  195.375000  196.523671  194.156402  194.925505  12046600   
3518 2024-12-27  192.538254  195.095322  190.430680  194.725737  18891400   
3519 2024-12-30  191.020004  192.328495  188.902433  189.581658  14264700   

          SMA_50      EMA_50     RSI_14      EMA_12  ...  shortTermDebt  \
3515  174.700717  176.990717  71.498729  188.083719  ...   2.887000e+09   
3516  175.326772  177.731646  70.668252  189.283823  ...   2.887000e+09   
3517  175.932663  178.423542  72.328084  190.220928  ...   2.887000e+09   
3518  176.487805  178.977060  67.337998  190.577439  ...   2.887000e+09   
3519  177.057080  179.449333  64.890216  190.645526  ...   2.887000e+09   

      longTermDebt  commonStockSharesOutstanding    netIncome       EPS  \
3515  1.088

In [17]:
merged_df.columns

Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'SMA_50', 'EMA_50',
       'RSI_14', 'EMA_12', 'EMA_26', 'MACD', 'SMA_20', 'STD_20', 'Upper_Band',
       'Lower_Band', 'High-Low', 'High-PrevClose', 'Low-PrevClose', 'TR',
       'ATR_14', 'OBV', 'totalAssets', 'totalCurrentAssets',
       'totalCurrentLiabilities', 'totalShareholderEquity', 'shortTermDebt',
       'longTermDebt', 'commonStockSharesOutstanding', 'netIncome', 'EPS',
       'Total_Debt', 'DE_Ratio', 'Current_Ratio', 'ROA', 'ROE'],
      dtype='object')

In [20]:
# Exporting to CSV
merged_df.to_csv('final-report/merged_df.csv', index=False)